In [3]:
from pandas import read_excel, to_datetime, DataFrame, read_sql_query
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from dateutil.parser import parse
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import os
from dotenv import load_dotenv
#import plotly.express as px
import openpyxl
import pymysql

from sqlalchemy import create_engine
from decouple import config 

In [14]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

True

In [15]:
query = '''
SELECT 
    dm.id_patient as id_patient,
    d.case_id,
    p.patient_code AS code,
    d.a_non_patisipan_an AS first_name,
    d.b_siyati AS last_name,
    TIMESTAMPDIFF(YEAR,
        d.nan_ki_dat_ou_fet,
        now()) AS age,
    d.nan_ki_dat_ou_fet AS dob,
    d.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS interview_date,
    d.e__telefn,
    d.d_adrs AS adress,
    IF(dm.id IS NOT NULL, 'yes', 'no') AS already_in_a_group,
    dm.id_group AS actual_id_group,
    dg.name AS actual_group_name,
    dm.id_parenting_group AS actual_id_parenting_group,
    dpg.name AS actual_parenting_group_name,
    dh.name AS actual_hub,
    ld.name AS actual_departement,
    d.f_komin AS commune,
    d.g_seksyon_kominal AS commune_section,
    d.b1_non_moun_mennen_entvyou_a AS interviewer_firstname,
    d.c1_siyati_moun_ki_f_entvyou_a AS interviewer_lastname,
    d.d1_kad AS interviewer_role,
    d.lot_kad AS interviewer_other_info,
    d.h_kote_entvyou_a_ft AS interview_location,
    d.paran_ou_vivan AS is_your_parent_alive,
    d.i_non_manman AS mothers_name,
    d.j_non_papa AS fathers_name,
    d.k_reskonsab_devan_lalwa AS who_is_your_law_parent,
    d.total,
    d.organisation,
    d.form_link
FROM
    caris_db.dreams_surveys_data d
        LEFT JOIN
    dream_member dm ON dm.case_id = d.case_id
        LEFT JOIN
    patient p ON p.id = dm.id_patient
        LEFT JOIN
    dream_group dg ON dg.id = dm.id_group
        LEFT JOIN
    dream_group dpg ON dpg.id = dm.id_parenting_group
        LEFT JOIN
    dream_hub dh ON dh.id = dg.id_dream_hub
        LEFT JOIN
    lookup_commune lc ON lc.id = dh.commune
        LEFT JOIN
    lookup_departement ld ON ld.id = lc.departement
'''


In [16]:
sdata= read_sql_query(query,engine,parse_dates=True)
#sdata.id_patient = sdata.id_patient.astype(int16)
#sdata.age = sdata.age.astype(int16)
SDATA = sdata[sdata.total>=14]

# close the pool of connection
engine.dispose()

In [5]:
#Connecting to Commcare
load_dotenv('id_cc.env')
email = os.getenv('COMCARE_EMAIL')
password_cc = os.getenv('COMCARE_PASSWORD')

#Defining the driver
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.implicitly_wait(1000)

#Creating login function
def dreams_schooling():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/ae3ce02aad63402d0108435a413d38cb/'
    )
    #driver.find_element_by_xpath('//*[@id="id_auth-username"]').send_keys(email)
    driver.find_element(By.XPATH,'//*[@id="id_auth-username"]').send_keys(email)
    #driver.find_element_by_xpath('//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.XPATH,'//*[@id="id_auth-password"]').send_keys(password_cc)
    driver.find_element(By.CSS_SELECTOR,'button[type=submit]').click()

#Muso beneficiaries
dreams_schooling()

#Download the database "All gardens"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,"//*[@id='download-export-form']/form/div[2]/div/div[2]/div[1]/button/span[1]").click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()    
driver.find_element(By.XPATH,"//*[@id='download-progress']/div/div/div[2]/div[1]/form/a/span[1]").click()    

#-----------------------------------------------------------------------------------------------------------------------

#Household count
def dreams_schooling_household():
    driver.get(
        'https://www.commcarehq.org/a/caris-test/data/export/custom/new/case/download/0202f95655479a37cb7c8cc4f5a9ae8c/'
    )
    
dreams_schooling_household()

#Download the database "Household count"
#driver.find_element_by_xpath('//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-export-form"]/form/div[2]/div/div[2]/div[1]/button/span[1]').click()
#driver.find_element_by_xpath('//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()
driver.find_element(By.XPATH,'//*[@id="download-progress"]/div/div/div[2]/div[1]/form/a/span[1]').click()

True

In [4]:
schooling_dreams = read_excel(f"~/Downloads/schooling_dreams {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")
dreams_schooling_household = read_excel(f"~/Downloads/dreams_schooling_household {str(datetime.today().strftime('%Y-%m-%d'))}.xlsx")

## Schooling DREAMS

In [17]:
schooling_dreams.rename(
    columns={
        'dreams_code':"code"
    },
    inplace=True
)
dreams_schooling_household.rename(
    columns={
        'dreams_code':"code"
    },
    inplace=True
)

' dreams_schooling_household.rename(\n    columns={\n        \'dreams_code\':"code"\n    },\n    inplace=True\n) '

In [18]:
schooling_dreams = schooling_dreams.merge(SDATA[['code','age','dob']],on="code",how="left")

In [19]:
schooling_dreams = schooling_dreams[schooling_dreams.closed == False]
vouchers = schooling_dreams[schooling_dreams.is_schooling_vouchers_elligible=='1']
payed_schooling_dreams = schooling_dreams[schooling_dreams.eskew_peye == "1"]
vouchers_payed = payed_schooling_dreams[payed_schooling_dreams.is_schooling_vouchers_elligible=='1']

In [20]:
schooling_dreams.caseid.count()
vouchers.caseid.count()
payed_schooling_dreams.caseid.count()

162

80

153

In [8]:
from datetime import datetime
from numpy import int16

def get_age(date):
    return datetime.now().year - date.year

def tranche_age_classique(age):
    """age classique"""
    if age>=10 and age<=14:
        return "10-14"
    elif age>=15 and age<=19:
        return "15-19"
    elif age>=20 and age<=24:
        return "20-24"
    elif age>=25 and age<=29:
        return "25-29"
    else:
        return "not_valid_age"
        
def tranche_age_mineur_majeur(age):
    """minor and major age"""
    if age>=10 and age<=17:
        return "10-17"
    elif age>=18 and age<=19:
        return "18-19"
    elif age>=20 and age<=24:
        return "20-24"
    elif age>=25 and age<=29:
        return "25-29"
    else:
        return "not_valid_age"
    
def ovc_age(age):
    """ovc valid age"""
    if age>=10 and age<=14:
        return "10-14"
    if age>=15 and age<=17:
        return "15-17"
    elif age>=18 and age<=24:
        return "not_ovc_age"
    else:
        return "not_valid_age"

In [21]:
payed_schooling_dreams.dob = to_datetime(payed_schooling_dreams.dob,errors="coerce")

In [22]:
payed_schooling_dreams.loc[:,'tranche_age'] = payed_schooling_dreams.loc[:,"age"].apply(lambda age: tranche_age_classique(age))
payed_schooling_dreams.loc[:,'tranche_age_min_maj'] = payed_schooling_dreams.loc[:,"age"].apply(lambda age: tranche_age_mineur_majeur(age))
payed_schooling_dreams.loc[:,'tranche_age_ovc'] = payed_schooling_dreams.loc[:,"age"].apply(lambda age: ovc_age(age))


In [23]:
payed_schooling_dreams.pivot_table(
    values="caseid",
    index="tranche_age",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)
payed_schooling_dreams.pivot_table(
    values="caseid",
    index="tranche_age_min_maj",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)

,caseid
tranche_age,
10-14,90
15-19,59
20-24,4
Total,153


,caseid
tranche_age_min_maj,
10-17,135
18-19,14
20-24,4
Total,153


In [24]:
payed_schooling_dreams.caseid.count()
vouchers_payed.caseid.count()

153

80

In [29]:
payed_schooling_dreams.to_excel('./payed_schooling_dreams.xlsx',index=False)

## Household
---



In [15]:
dreams_schooling_household.caseid.count()

234

In [16]:
dreams_schooling_household = dreams_schooling_household[dreams_schooling_household.closed==False]
dreams_schooling_household.nunique()

number                                        229
caseid                                        229
name                                           48
arv                                             2
commune                                         4
dob                                            74
code                                           80
est-ce_que_cette_personne_est_un_caregiver      2
office                                          1
often_sick                                      2
positif                                         3
sexe                                            2
test                                            2
indices.schooling_dreams                       80
closed                                          1
closed_by_username                              1
closed_date                                     1
last_modified_by_user_username                  7
last_modified_date                             81
opened_by_username                              7


In [17]:
print('direct beneficiaire')
dreams_schooling_household.code.nunique()

direct beneficiaire


80

In [18]:
directbene = DataFrame()
directbene['code'] = dreams_schooling_household.code.drop_duplicates()
directbene = directbene.merge(SDATA[['code','age']],on="code",how='left')

directbene.loc[:,'tranche_age'] = directbene.loc[:,"age"].apply(lambda age: tranche_age_classique(age))
directbene.loc[:,'tranche_age_min_maj'] = directbene.loc[:,"age"].apply(lambda age: tranche_age_mineur_majeur(age))
directbene.loc[:,'tranche_age_ovc'] = directbene.loc[:,"age"].apply(lambda age: ovc_age(age))

In [19]:
directbene.pivot_table(
    values="code",
    index="tranche_age_ovc",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)

directbene.pivot_table(
    values="code",
    index="tranche_age_min_maj",
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name="Total"
)

,code
tranche_age_ovc,
10-14,49
15-17,22
not_ovc_age,9
Total,80


,code
tranche_age_min_maj,
10-17,71
18-19,6
20-24,3
Total,80


In [20]:
caregiver = dreams_schooling_household[dreams_schooling_household['est-ce_que_cette_personne_est_un_caregiver']==1]
caregiver.caseid.count()

80

In [21]:
print('indirect beneficiaire')
ib = dreams_schooling_household[dreams_schooling_household['est-ce_que_cette_personne_est_un_caregiver']==0]
ib.caseid.count()

indirect beneficiaire


148

## temp export

In [22]:
caregiver.to_excel('caregiver_dreams_schooling_household.xlsx',index=False)
ib.to_excel('ib_dreams_schooling_household.xlsx',index=False)
directbene.to_excel('directbene.xlsx',index=False)

In [23]:
driver.quit()

## Any Missing

In [24]:
#vouchers_payed[~vouchers_payed.code.isin(directbene.code)].to_excel('missing_comptage.xlsx',index=False)